# Multi-Agent Article Generation: Native A2A Deployment and Source-Based Orchestration on Vertex AI Agent Engine

This document provides a technical overview and step-by-step guide for deploying a multi-agent system on **Vertex AI Agent Engine**. It specifically addresses two critical deployment patterns: using **native Agent-to-Agent (A2A) support** for child agents and the **"deploy from source files"** method for complex root agents.

---

## Executive Summary

When building sophisticated AI systems, a single agent often isn't enough. You need a hierarchy where a "Root Agent" orchestrates several "Child Agents." This notebook demonstrates how to:

1. **Leverage Native A2A Support:** Deploy individual agents that can communicate via the standardized A2A protocol.
2. **Bypass Serialization Limits:** Use the "source-based deployment" feature to deploy agents that contain non-serializable objects (like network clients or complex class instances), which would otherwise fail using standard cloudpickle serialization.

---

## Deployment Architecture

The system consists of a **Root Agent** (Article Generation Flow) that manages the lifecycle of a request by delegating tasks to specialized **Child Agents**:

* **Research Agents (1 & 2):** Handle topic selection and report generation.
* **Writer Agent:** Drafts the article based on research.
* **Reviewer Agent:** Provides editorial feedback and improvement points.

---

## Phase 1: Deploying A2A-Enabled Child Agents

The child agents are deployed using the native `A2aAgent` class in `vertexai`. This wraps a standard `LlmAgent` from the **Agent Development Kit (ADK)** and exposes it via the A2A protocol.

### Key Implementation Details

* **A2aAgentExecutor:** Each child agent uses an executor that wraps its logic to handle A2A-specific communication.
* **Session Management:** A custom `MyVertexAiSessionService` is implemented to ensure session persistence across the Vertex AI infrastructure.
* **Direct Deployment:** These agents are small enough to be deployed directly using `client.agent_engines.create(agent=a2a_agent, ...)`.

---

## Phase 2: Deploying the Root Agent from Source

The Root Agent is more complex. It contains `RemoteA2aAgent` instances and `httpx.AsyncClient` objects which are **not serializable** via standard methods. To solve this, we deploy from source files.

### 1. Structure the Source Directory

Create a dedicated directory (e.g., `article_agent/`) containing your logic. The `agent.py` file defines the `AdkApp` entry point.

### 2. Define the Entry Point

Instead of passing a live object to the deployment function, you define the agent inside the script. The `AdkApp` object acts as the interface for Agent Engine:

```python
# Inside article_agent/agent.py
app = AdkApp(
    agent=root_agent,
    app_name=os.environ.get('GOOGLE_CLOUD_AGENT_ENGINE_ID'),
)

```

### 3. Deploy via Configuration

Using `vertexai.Client`, specify the source package and the module path to the `AdkApp` object. This tells Agent Engine to build the environment from your files rather than serializing your local memory state.

```python
config = {
    'display_name': 'article_agent',
    'source_packages': ['article_agent'],
    'entrypoint_module': 'article_agent.agent',
    'entrypoint_object': 'app',
    'requirements_file': 'article_agent/requirements.txt',
    # ...
}
remote_agent = client.agent_engines.create(config=config)

```

---

## Critical Considerations

* **Authentication:** When the Root Agent calls A2A child agents, it requires a refreshable auth token. The provided `GoogleAuthRefresh` class (inheriting from `httpx.Auth`) ensures the agent can always authenticate with Google Cloud APIs.
* **Environment Variables:** Agent Engine automatically injects `GOOGLE_CLOUD_AGENT_ENGINE_ID`. Use this as your `app_name` in the `AdkApp` to maintain consistent session mapping.
* **Dependency Matching:** Ensure the `requirements.txt` in your source folder exactly matches the versions used during local development to avoid `ModuleNotFoundError` during remote execution.

## Package installation

In [ ]:
%pip install --user \
    google-adk==1.21.0 \
    google-cloud-aiplatform==1.132.0 \
    google-genai==1.56.0 \
    a2a-sdk==0.3.22

In [4]:
# restart the kernel.
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

## Preparation

In [2]:
import httpx, json, os, uuid
from google.auth import default
from google.auth.transport.requests import Request
import vertexai
from vertexai.agent_engines import AdkApp

# ADK
from google.genai.types import Part, Content, HttpOptions
from google.adk.agents.callback_context import CallbackContext
from google.adk.agents.llm_agent import LlmAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.adk.agents.sequential_agent import SequentialAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.models import LlmResponse, LlmRequest
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, VertexAiSessionService

# A2A
from a2a.client import ClientConfig, ClientFactory
from a2a.types import AgentCard, TransportProtocol
from google.adk.a2a.executor.a2a_agent_executor import A2aAgentExecutor
from google.adk.a2a.utils.agent_card_builder import AgentCardBuilder

# Agent Engine
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import A2aAgent
from vertexai.preview.reasoning_engines.templates.a2a import create_agent_card


[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
[PROJECT_NUMBER] = !gcloud projects describe {PROJECT_ID} --format='value(projectNumber)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION)

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

In [3]:
# Chat client to test AdkApp
class ChatClient:
    def __init__(self, app, user_id='default_user'):
        self._app = app
        self._user_id = user_id
        self._session_id = None

    async def async_stream_query(self, message):
        if not self._session_id:
            session = await self._app.async_create_session(
                user_id=self._user_id,
            )
            self._session_id = getattr(session, 'id', None) or session['id']

        result = []
        async for event in self._app.async_stream_query(
            user_id=self._user_id,
            session_id=self._session_id,
            message=message,
        ):
            if ('content' in event and 'parts' in event['content']):
                response = '\n'.join(
                    [p['text'] for p in event['content']['parts'] if 'text' in p]
                )
                if response:
                    print(response)
                    result.append(response)
        return result

## Define and deploy A2A agents.

In [4]:
instruction = '''
Your role is to collect information necessary for writing an article and summarize it into a research report.
Create a list of about 5 relevant topics to refer to when writing an article on a specified theme.
A downstream agent will create a research report based on this list.

* Output Format
Output in English.
'''

research_agent1 = LlmAgent(
    name='research_agent1',
    model='gemini-2.5-flash',
    description='Agent that collects info for writing articles (Topic Selection)',
    instruction=instruction,
)

In [5]:
instruction = '''
Your role is to collect information necessary for writing an article and summarize it into a research report.
An upstream agent will specify approximately 5 topics for investigation.

* Output Format
Output in English.
The research report should summarize objective information for each topic. For each topic, write at least 5 sentences.
'''

research_agent2 = LlmAgent(
    name='research_agent2',
    model='gemini-2.5-flash',
    description='Agent that collects info for writing articles (Report Creation)',
    instruction=instruction,
)

In [6]:
instruction = '''
Your role is to write a casual reading article on a specific theme.
You will be provided with an article "Theme" and an associated "Research Report."
Please write a reliable, readable article based on the objective facts described in the research report.

**Output Conditions**
- Assume a reader with some basic knowledge of the topic and keep the content light enough to be read in a few minutes.
- Use a relatively casual and conversational tone.
- Do not output the thought process; output only the final result.
- Do not provide an article title. Use the following structure. You may arrange the section titles according to the content:
  0. Introduction: Do not use a section title. Summarize the introduction in 1-2 sentences to make the reader want to read this article.
  1. Overview: Summarize and briefly explain the overall picture of the topic.
  2. Latest Information: Highlight new information of particular interest.
  3. Practice: Introduce one thing the reader could try themselves regarding the topic.
  4. Summary

- Use Markdown headers ## for each section title. Use bulleted Markdown where necessary.
- Do not use any other Markdown decorations.

**Revision based on Reviewer's Instructions**
- When a reviewer provides points for improvement, follow the instructions to modify the previous article without being strictly bound by the output conditions.
- Do not modify parts other than those indicated by the improvement points.
'''

writer_agent = LlmAgent(
    name='writer_agent',
    model='gemini-2.5-flash',
    description='Agent that writes reading articles on specific themes',
    instruction=instruction,
)

In [7]:
instruction = f'''
Your role is to review the reading article and provide improvement comments to ensure the content meets the article criteria.

* Article Criteria
- The article must have a title of about 40 characters at the beginning.
  The title should include useful life information that makes the reader feel "I must read this now."
  Use Markdown header # for the title.
- Immediately after the title, add an introduction summarizing "Why this theme is being addressed now" to motivate the reader.
- Use emojis in each section subtitle to create a friendly feel.
- At least 3 specific examples that the reader can practice today must be introduced.

* Output Format
- Output in English.
- First, explain the good points of the article.
- Next, output the correction points as a bulleted list.
'''

review_agent = LlmAgent(
    name='review_agent',
    model='gemini-2.5-flash',
    description='Agent that reviews reading articles',
    instruction=instruction,
)

In [8]:
# Wrapper class of VertexAiSessionService that's compatible with InMemorySessionService
class MyVertexAiSessionService(VertexAiSessionService):

    async def create_session(self, app_name, user_id, state={}, session_id=None):
        # Ignore session_id
        session = await super().create_session(
            app_name=app_name,
            user_id=user_id,
            state=state,
        )
        return session

    async def get_session(self, app_name, user_id, session_id):
        # Return None if session non exists.
        try:
            session = await super().get_session(
                app_name=app_name,
                user_id=user_id,
                session_id=session_id,
            )
            return session
        except:
            return None


def get_create_runner_class(agent):
    async def create_runner():
        return Runner(
            # Use resource_name as app_name for VertexAiSessionService.
            app_name=os.environ.get('GOOGLE_CLOUD_AGENT_ENGINE_ID'),
            agent=agent,
            artifact_service=InMemoryArtifactService(),
            session_service=MyVertexAiSessionService(),
            memory_service=InMemoryMemoryService(),
        )
    return create_runner


def get_agent_executor_class(agent):
    def agent_executor_builder():
        return A2aAgentExecutor(
            runner=get_create_runner_class(agent),
        )       
    return agent_executor_builder


async def get_agent_card(agent):
    builder = AgentCardBuilder(agent=agent)
    adk_agent_card = await builder.build()
    return create_agent_card(
        agent_name=adk_agent_card.name,
        description=adk_agent_card.description,
        skills=adk_agent_card.skills
)

In [9]:
agents = {
    'research_agent1_a2a': research_agent1,
    'research_agent2_a2a': research_agent2,
    'writer_agent_a2a': writer_agent,
    'review_agent_a2a': review_agent,
}

def get_agent_resource(agent_name):
    for agent in agent_engines.list():
        if agent.display_name == agent_name:
            return agent.resource_name
    return None

client = vertexai.Client(
    project=PROJECT_ID,
    location=LOCATION,
    http_options=HttpOptions(
        api_version='v1beta1', base_url=f'https://{LOCATION}-aiplatform.googleapis.com/'
    ),
)

for agent_name, agent in agents.items():
    print(f'Deploying {agent_name}...')
    a2a_agent = A2aAgent(
        agent_card=await get_agent_card(agent),
        agent_executor_builder=get_agent_executor_class(agent) # , None)
    )
    config={
        'display_name': agent_name,
        'description': a2a_agent.agent_card.description,
        'requirements': [
            'google-adk==1.21.0',
            'google-cloud-aiplatform==1.132.0',
            'google-genai==1.56.0',
            'a2a-sdk==0.3.22',
        ],
        'http_options': {
            'base_url': f'https://{LOCATION}-aiplatform.googleapis.com',
            'api_version': 'v1beta1',
        },
        'staging_bucket': f'gs://{PROJECT_ID}',
    }

    resource_name = get_agent_resource(agent_name)
    if not resource_name:
        # Deploy agent
        remote_a2a_agent = client.agent_engines.create(
            agent=a2a_agent,
            config=config,
        )
    else:
        # Update agent
        remote_a2a_agent = client.agent_engines.update(
            name=resource_name,
            agent=a2a_agent,
            config=config,
        )

Deploying research_agent1_a2a...


/var/tmp/ipykernel_56820/1364834990.py:48: UserWarning: [EXPERIMENTAL] AgentCardBuilder: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  builder = AgentCardBuilder(agent=agent)
The following requirements are missing: {'pydantic', 'cloudpickle', 'a2a'}


Deploying research_agent2_a2a...


/var/tmp/ipykernel_56820/1364834990.py:48: UserWarning: [EXPERIMENTAL] AgentCardBuilder: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  builder = AgentCardBuilder(agent=agent)
The following requirements are missing: {'pydantic', 'cloudpickle', 'a2a'}


Deploying writer_agent_a2a...


The following requirements are missing: {'pydantic', 'cloudpickle', 'a2a'}


Deploying review_agent_a2a...


The following requirements are missing: {'pydantic', 'cloudpickle', 'a2a'}


## Create source files for root agent, and deploy from source files.

In [11]:
%%bash
mkdir -p article_agent
cat <<'EOF' >article_agent/agent.py
import httpx, os
from google.auth import default
from google.auth.transport.requests import Request
from vertexai.agent_engines import AdkApp

# ADK
from google.genai.types import Part, Content
from google.adk.agents.callback_context import CallbackContext
from google.adk.agents.llm_agent import LlmAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.adk.agents.sequential_agent import SequentialAgent
from google.adk.models import LlmResponse, LlmRequest

# A2A
from a2a.client import ClientConfig, ClientFactory
from a2a.types import TransportProtocol
from google.adk.a2a.executor.a2a_agent_executor import A2aAgentExecutor

class GoogleAuthRefresh(httpx.Auth):
    def __init__(self, scopes):
        self.credentials, _ = default(scopes=scopes)
        self.transport_request = Request()
        self.credentials.refresh(self.transport_request)

    def auth_flow(self, request):
        if not self.credentials.valid:
            self.credentials.refresh(self.transport_request)
        request.headers['Authorization'] = f'Bearer {self.credentials.token}'
        yield request


factory = ClientFactory(
    ClientConfig(
        supported_transports=[TransportProtocol.http_json],
        use_client_preference=True,
        httpx_client=httpx.AsyncClient(
            timeout=60,
            headers={'Content-Type': 'application/json'},
            auth=GoogleAuthRefresh(scopes=['https://www.googleapis.com/auth/cloud-platform']) 
        ),
    ),
)

#resource_name = get_agent_resource('research_agent1_a2a')
a2a_url = f'https://__LOCATION__-aiplatform.googleapis.com/v1beta1/__research_agent1_a2a__/a2a'
research_agent1_remoteA2a = RemoteA2aAgent(
    name='research_agent1',
    description='Agent that collects info for writing articles (Topic Selection)',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

#resource_name = get_agent_resource('research_agent2_a2a')
a2a_url = f'https://__LOCATION__-aiplatform.googleapis.com/v1beta1/__research_agent2_a2a__/a2a'
research_agent2_remoteA2a = RemoteA2aAgent(
    name='research_agent2',
    description='Agent that collects info for writing articles (Report Creation)',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

#resource_name = get_agent_resource('writer_agent_a2a')
a2a_url = f'https://__LOCATION__-aiplatform.googleapis.com/v1beta1/__writer_agent_a2a__/a2a'
writer_agent_remoteA2a = RemoteA2aAgent(
    name='writer_agent',
    description='Agent that writes reading articles on specific themes',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

#resource_name = get_agent_resource('review_agent_a2a')
a2a_url = f'https://__LOCATION__-aiplatform.googleapis.com/v1beta1/__review_agent_a2a__/a2a'
review_agent_remoteA2a = RemoteA2aAgent(
    name='review_agent',
    description='Agent that reviews reading articles',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

def get_print_agent(text):
    def before_model_callback(
        callback_context: CallbackContext, llm_request: LlmRequest
    ) -> LlmResponse:
        return LlmResponse(
            content=Content(
                role='model', parts=[Part(text=text)],
            )
        )
    return LlmAgent(
        name='print_agent',
        model='gemini-2.5-flash', # not used
        description='',
        instruction = '',
        before_model_callback=before_model_callback,
    )

research_agent = SequentialAgent(
    name='research_agent',
    sub_agents=[
        get_print_agent('\n---\n## The Research Agent will create a research report.\n---\n'),
        get_print_agent('\n## Selecting topics for investigation.\n'),
        research_agent1_remoteA2a,
        get_print_agent('\n## Creating a research report based on the selected topics.\n'),
        research_agent2_remoteA2a,
        get_print_agent('\n#### The research report is ready. Shall I start creating the article?\n'),
    ],
    description='Agent that collects info for writing articles and summarizes it into a report',
)

write_and_review_agent = SequentialAgent(
    name='write_and_review_agent',
    sub_agents=[
        get_print_agent('\n---\n## The Writer Agent will write the article.\n---\n'),
        writer_agent_remoteA2a,
        get_print_agent('\n---\n## The Reviewer Agent will review the article.\n---\n'),
        review_agent_remoteA2a,
       get_print_agent('\n#### Would you like to request article revisions based on the review?\n'),
    ],
    description='Creates and reviews articles.',
)

root_agent = LlmAgent(
    name='article_generation_flow',
    model='gemini-2.5-flash',
    instruction = '''
If asked what you can do, explain the following processes in a clear and friendly manner.

- Execute a workflow to create an article on a theme specified by the user.
- First, create a research report on the theme.
- After that, the Writer Agent and Reviewer Agent work together to create an article in accordance with the editorial policy.

When a user specifies a theme for an article, execute the following flow:

1. State that you will begin creating the article for that theme, transfer to research_agent, and request a research report.
2. Once the user approves the article creation, transfer to write_and_review_agent to request article creation and review.
3. If the user wants to revise the article, transfer to write_and_review_agent.

**Conditions**
The nickname for research_agent is "Research Agent"
The nicknames for write_and_review_agent are "Writer Agent" and "Reviewer Agent"

''',
    sub_agents=[
        research_agent,
        write_and_review_agent,
    ],
    description='Agent that executes the workflow for creating articles'
)

# entrypoint_object
app = AdkApp(
    agent=root_agent,
    # app_name should be the full Reasoning Engine resource name.
    app_name=os.environ.get('GOOGLE_CLOUD_AGENT_ENGINE_ID', 'default-app-name'),
)
EOF

cat <<'EOF' >article_agent/requirements.txt
google-adk==1.21.0
google-cloud-aiplatform==1.132.0
google-genai==1.56.0
a2a-sdk==0.3.22
EOF

In [12]:
!sed -i 's/__LOCATION__/{LOCATION}/g' article_agent/agent.py
for a2a_agent in agents.keys():
    resource_name = get_agent_resource(a2a_agent)
    print(resource_name)
    !sed -i 's:__{a2a_agent}__:{resource_name}:g' article_agent/agent.py

projects/879055303739/locations/us-central1/reasoningEngines/6599239103097602048
projects/879055303739/locations/us-central1/reasoningEngines/5230144816376971264
projects/879055303739/locations/us-central1/reasoningEngines/4824820849913626624
projects/879055303739/locations/us-central1/reasoningEngines/8808254720322830336


In [13]:
DISPLAY_NAME = 'article_agent'

client = vertexai.Client(
    project=PROJECT_ID,
    location=LOCATION,
    http_options=HttpOptions(
        api_version='v1beta1',
        base_url=f'https://{LOCATION}-aiplatform.googleapis.com/'
    ),
)

config = {
        'display_name': DISPLAY_NAME,
        'source_packages': ['article_agent'],
        'entrypoint_module': 'article_agent.agent',
        'entrypoint_object': 'app',
        'requirements_file': 'article_agent/requirements.txt',
        'class_methods': [
            {
                'name': 'async_stream_query',
                'api_mode': 'async_stream',
                'description': 'Stream responses from the agent for a given query.',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'input': {'type': 'string', 'description': 'The user query'},
                        'config': {'type': 'object', 'description': 'Optional runtime config'}
                    },
                    'required': ['input']
                }
            },
            {
                'name': 'async_create_session',
                'api_mode': 'async',
                'description': 'Create a new managed session.'
            }
        ]
}

remote_agent = client.agent_engines.create(
    config=config,
)

## Test the deployed agent

In [14]:
remote_agent = client.agent_engines.get(name=get_agent_resource(DISPLAY_NAME))
remote_agent

AgentEngine(api_resource.name='projects/879055303739/locations/us-central1/reasoningEngines/8209275969882554368')

In [15]:
chat_client = ChatClient(remote_agent)
_ = await chat_client.async_stream_query('What can you do?')

I can help you create an article on a theme you specify. Here's how it works:

1.  **Research Report:** First, I'll transfer your request to the **Research Agent**, who will create a detailed research report on your chosen theme.
2.  **Article Creation and Review:** Once you approve the research, I'll transfer to the **Writer Agent** and **Reviewer Agent**. They will work together to write the article according to an editorial policy and ensure its quality.
3.  **Revisions:** If you'd like any changes to the article, I can also transfer to the **Writer Agent** and **Reviewer Agent** to make those revisions.

So, in short, I can manage the entire process of generating an article for you, from initial research to final review.


In [16]:
_ = await chat_client.async_stream_query('Create an article with the theme "New ways to enjoy Hanami (Cherry Blossom Viewing)"')

I will begin creating an article with the theme "New ways to enjoy Hanami (Cherry Blossom Viewing)". First, I will transfer to the Research Agent to gather information and create a research report on this topic.

---
## The Research Agent will create a research report.
---


## Selecting topics for investigation.

Here are about 5 relevant topics to refer to when writing an article on "New ways to enjoy Hanami (Cherry Blossom Viewing)":

1.  **Digital and Virtual Hanami Experiences:** Exploring how technology (e.g., online livestreams, VR/AR experiences, social media trends like themed filters or challenges) is creating new ways to appreciate cherry blossoms, especially for those who cannot attend in person.
2.  **Unique Hanami Spots and Alternative Viewing Activities:** Highlighting unconventional locations for hanami (e.g., historical sites, urban rooftops, lesser-known local parks, specific trails) and non-traditional activities beyond picnicking (e.g., cycling tours, boat cruises, 

In [17]:
_ = await chat_client.async_stream_query('Please proceed.')


---
## The Writer Agent will write the article.
---

Cherry blossom season is a magical time, but did you know there are more ways to enjoy hanami than ever before? This beloved tradition is evolving, offering exciting new experiences for everyone.

## Overview

Hanami, or cherry blossom viewing, is no longer just about spreading a blanket under the trees for a picnic, though that remains a cherished classic! Today, the experience is expanding in incredible ways. You can find unique viewing spots like urban rooftops offering panoramic cityscapes dotted with pink, or enjoy serene boat cruises along rivers lined with blossoms. Even activities like cycling tours through flowery avenues are becoming popular. Beyond just looking, hanami now often includes immersive themed events, artistic light installations that transform night scenes into a dream, and cultural performances right under the blooming canopy. And let's not forget the culinary delights – chefs are getting creative with sakura

In [18]:
_ = await chat_client.async_stream_query('Please proceed.')


---
## The Writer Agent will write the article.
---

# 🌸 Hanami Evolved: Uncover New Ways to Celebrate Spring!

Cherry blossom season is here, but what if you're looking for more than just a traditional picnic under the trees? This year, Hanami is blossoming with exciting new possibilities, blending timeless tradition with modern twists for everyone to enjoy.

## 🌸 Hanami's New Horizon

Hanami, the cherished Japanese custom of appreciating cherry blossoms, is wonderfully adaptable. While gathering under a canopy of pink petals remains a beloved pastime, the way we celebrate is continually evolving. Forget just finding a spot in a crowded park; today's Hanami embraces a broader spectrum of experiences, from quiet contemplation to vibrant, themed events. It's about making the most of this fleeting beauty in ways that suit your lifestyle and values.

## ✨ What's Blooming Now: Latest Trends

The world of Hanami is bursting with fresh ideas! Here are some of the most exciting developments 

In [19]:
_ = await chat_client.async_stream_query('It is complete now. Thank you.')

You're welcome! I'm glad I could assist in generating the article "🌸 Hanami Evolved: Uncover New Ways to Celebrate Spring!".

The article generation process is now complete.
